- https://abicky.net/2015/03/08/120241/
- https://www.aclweb.org/anthology/C96-1009

In [1]:
import pandas as pd
import numpy as np

ndict = {
    'Staff Reporter of The Wall Street Journal': 19,
    'Wall Street Journal': 26,
    'The Wall Street Journal': 22,
    'Wall Street': 38,
    'The Wall Street': 23,
    'Wall Street analysts': 3,
    'of The Wall Street Journal': 20,
    'Reporter of The Wall Street': 19,
    'Reporter of The Wall Street Journal': 19,
    'Staff Reporter of The Wall Street': 19,
    'of The Wall Street': 20
}

df = pd.DataFrame({
    'phrase': list(ndict.keys()),
    'n(a)': list(ndict.values())
})

In [2]:
df

,phrase,n(a)
0,Staff Reporter of The Wall Street Journal,19
1,Wall Street Journal,26
2,The Wall Street Journal,22
3,Wall Street,38
4,The Wall Street,23
5,Wall Street analysts,3
6,of The Wall Street Journal,20
7,Reporter of The Wall Street,19
8,Reporter of The Wall Street Journal,19
9,Staff Reporter of The Wall Street,19


In [3]:
df['|a|'] = [len(v) for v in df['phrase'].str.split()]
df['t(a)'] = [0 for _ in df['phrase']]
df['c(a)'] = [0 for _ in df['phrase']]
df['c-value'] = [np.nan for _ in df['phrase']]
df = df.sort_values('|a|', ascending=False).reset_index(drop=True)

In [4]:
df

,phrase,n(a),|a|,t(a),c(a),c-value
0,Staff Reporter of The Wall Street Journal,19,7,0,0,NaN
1,Reporter of The Wall Street Journal,19,6,0,0,NaN
2,Staff Reporter of The Wall Street,19,6,0,0,NaN
3,of The Wall Street Journal,20,5,0,0,NaN
4,Reporter of The Wall Street,19,5,0,0,NaN
5,The Wall Street Journal,22,4,0,0,NaN
6,of The Wall Street,20,4,0,0,NaN
7,Wall Street Journal,26,3,0,0,NaN
8,The Wall Street,23,3,0,0,NaN
9,Wall Street analysts,3,3,0,0,NaN


In [5]:
max_a = max(df['|a|'])
max_a

7

In [6]:
def revise_all_substrs():
    for i, b in enumerate(df['phrase']):
        if ((b in row['phrase']) and (b != row['phrase'])):
            df.loc[i, 't(a)'] += (row['n(a)'] - row['t(a)'])
            if (row['n(a)'] - row['t(a)']) > 0:
                df.loc[i, 'c(a)'] += 1

def calc_c_value(row):
    for i, r in df[df['|a|'] > row['|a|']].iterrows():
        if row['n(a)'] == r['n(a)']:
            return 0
    if row['c(a)'] == 0:
        return (row['|a|'] - 1) * row['n(a)']
    else:
        return (row['|a|'] - 1) * (row['n(a)'] - row['t(a)'] / row['c(a)'])


In [7]:
for index in df[df['|a|'] == max_a].index:
    row = df.loc[index, :]
    df.loc[index, 'c-value'] = (max_a - 1) * row['n(a)']
    revise_all_substrs()

In [8]:
df

,phrase,n(a),|a|,t(a),c(a),c-value
0,Staff Reporter of The Wall Street Journal,19,7,0,0,114.0
1,Reporter of The Wall Street Journal,19,6,19,1,NaN
2,Staff Reporter of The Wall Street,19,6,19,1,NaN
3,of The Wall Street Journal,20,5,19,1,NaN
4,Reporter of The Wall Street,19,5,19,1,NaN
5,The Wall Street Journal,22,4,19,1,NaN
6,of The Wall Street,20,4,19,1,NaN
7,Wall Street Journal,26,3,19,1,NaN
8,The Wall Street,23,3,19,1,NaN
9,Wall Street analysts,3,3,0,0,NaN


In [9]:
for index in df[df['|a|'] != max_a].ind():
    row = df.loc[index, :]
    c_value = calc_c_value(row)
    df.loc[index, 'c-value'] = c_value
    
    if (c_value != 0):
        revise_all_substrs()

In [10]:
df

,phrase,n(a),|a|,t(a),c(a),c-value
0,Staff Reporter of The Wall Street Journal,19,7,0,0,114.000000
1,Reporter of The Wall Street Journal,19,6,19,1,0.000000
2,Staff Reporter of The Wall Street,19,6,19,1,0.000000
3,of The Wall Street Journal,20,5,19,1,4.000000
4,Reporter of The Wall Street,19,5,19,1,0.000000
5,The Wall Street Journal,22,4,20,2,36.000000
6,of The Wall Street,20,4,20,2,0.000000
7,Wall Street Journal,26,3,22,3,37.333333
8,The Wall Street,23,3,22,3,31.333333
9,Wall Street analysts,3,3,0,0,6.000000


In [11]:
df.sort_values('c-value', ascending=False).reset_index(drop=True)

,phrase,n(a),|a|,t(a),c(a),c-value
0,Staff Reporter of The Wall Street Journal,19,7,0,0,114.000000
1,Wall Street Journal,26,3,22,3,37.333333
2,The Wall Street Journal,22,4,20,2,36.000000
3,Wall Street,38,2,30,6,33.000000
4,The Wall Street,23,3,22,3,31.333333
5,Wall Street analysts,3,3,0,0,6.000000
6,of The Wall Street Journal,20,5,19,1,4.000000
7,Reporter of The Wall Street Journal,19,6,19,1,0.000000
8,Staff Reporter of The Wall Street,19,6,19,1,0.000000
9,Reporter of The Wall Street,19,5,19,1,0.000000
